- NLP w/ RNN: Movie review
- Florent Dufour
- 2016 - 2022

# Sentiment analysis

- Movie review dataset from IMDB

In [10]:
import tensorflow as tf
import keras
import numpy as np
import os
from   keras.datasets import imdb
from   keras.preprocessing import sequence

VOCAB_SIZE = 88584
MAX_LEN    = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_label) = imdb.load_data(num_words = VOCAB_SIZE)

In [2]:
train_data

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1

In [3]:
# Pre-processing
# All input text should be the same length for the model
# Padding: Add empty words at the beginning of the sentence to fit MAX_LEN
train_data = sequence.pad_sequences(train_data, MAX_LEN)
test_data = sequence.pad_sequences(test_data, MAX_LEN)

In [4]:
# Create the model
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(VOCAB_SIZE, 32),
  tf.keras.layers.LSTM(32),
  tf.keras.layers.Dense(1, activation="sigmoid") # <0.5: Negative / >0.5 Positive
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Training
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"])
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2) # Validation w/ 20% of the data

Epoch 1/10
625/625 [==============================] - 42s 57ms/step - loss: 0.4210 - acc: 0.8095 - val_loss: 0.3543 - val_acc: 0.8632
Epoch 2/10
625/625 [==============================] - 35s 55ms/step - loss: 0.2446 - acc: 0.9059 - val_loss: 0.3081 - val_acc: 0.8666
Epoch 3/10
625/625 [==============================] - 35s 56ms/step - loss: 0.1867 - acc: 0.9330 - val_loss: 0.2740 - val_acc: 0.8906
Epoch 4/10
625/625 [==============================] - 35s 56ms/step - loss: 0.1554 - acc: 0.9444 - val_loss: 0.2777 - val_acc: 0.8876
Epoch 5/10
625/625 [==============================] - 35s 55ms/step - loss: 0.1316 - acc: 0.9540 - val_loss: 0.3125 - val_acc: 0.8918
Epoch 6/10
625/625 [==============================] - 34s 55ms/step - loss: 0.1163 - acc: 0.9592 - val_loss: 0.3104 - val_acc: 0.8866
Epoch 7/10
625/625 [==============================] - 34s 54ms/step - loss: 0.1030 - acc: 0.9653 - val_loss: 0.3487 - val_acc: 0.8908
Epoch 8/10
625/625 [==============================] - 34s 55ms

In [7]:
results = model.evaluate(test_data, test_label)
print(results)

782/782 [==============================] - 13s 16ms/step - loss: 0.4259 - acc: 0.8430
[0.4259374737739563, 0.8429999947547913]


In [13]:
# Making predictions

word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAX_LEN)[0]

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

In [14]:
positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

[0.8131393]
[0.30418083]
